<a href="https://colab.research.google.com/github/leomiquelutti/razorback/blob/master/docs/source/tutorials/survey_study_colab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Survey Study: a tutorial

This Jupyter notebook aims to help razorback users to compute impedance estimates from the data set shown in the paper in different ways for a two stage remote reference configuration:

1- Ordinary Least Squares

2- M-Estimator 

3- Bounded Influence 

This tutorial is designed for Metronix data format (.ats files).



## Preparing the environment

After you run the cell below, click on `RESTART RUNTIME` before moving in within the notebook. When you restart the runtime, you allow for the changes on the upgraded `matplotlib` package to take effect. 

**MAKE SURE YOU RESTART THE RUNTIME BEFORE MOVING ON**

In [ ]:
!pip install razorback
!pip install gitpython
!pip install --upgrade matplotlib

Now that you restarted the runtime, go to the next section to download the data.

## Downloading the data

The cells below both:
* download the data to your environment
* put the calibration files in the proper place

In [ ]:
# download the data
import git
git.Git("/content").clone("git://github.com/BRGM/razorback-tutorial-data.git")

In [ ]:
# put a folder of the calibration files in the right place
!mkdir /usr/local/lib/python3.7/dist-packages/razorback/data/
!cp -r /content/razorback-tutorial-data/metronix_calibration /usr/local/lib/python3.7/dist-packages/razorback/data/

## Data processing

In [ ]:
import razorback as rb # importing the razorback library
import numpy as np # importing the numpy library as np
import matplotlib.pyplot as plt # importing the matplotlib.pyplot library as plt
import urllib.request
import glob
%matplotlib inline

In [ ]:
# function for getting sensor information from .ats file header
def sensor(ats_file):
    header = rb.io.ats.read_ats_header(ats_file) #razorback function for ats file data importation
    chan = header['channel_type'].decode()
    stype = ''.join(c for c in header['sensor_type'].decode() if c.isprintable())
    snum = header['sensor_serial_number']
    sampling_rate = header['sampling_rate']
    x1, y1, z1 = header['x1'], header['y1'], header['z1']
    x2, y2, z2 = header['x2'], header['y2'], header['z2']
    L = ((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)**.5
    return chan, L, stype, snum, sampling_rate

In [ ]:
# function for getting calibration function from sensor information
def calibration(ats_file, name_converter=None):
    chan, L, stype, snum, sampling_rate = sensor(ats_file)
    if chan in ('Ex', 'Ey'):
        return L
    elif chan in ('Hx', 'Hy', 'Hz'):
        calib_name = f"{stype}{snum:03d}.txt"
        if name_converter:
            calib_name = name_converter.get(calib_name, calib_name)
        return rb.calibrations.metronix(calib_name, sampling_rate)
    raise Exception(f"Unknown channel name: {chan}")

The functions `sensor()` and `calibration()` will help to load the metadata of the data set (electric dipoles length, orientation, calibration files...).

Other strategies for handling these metadata could be used, it's up to you to design your own.


In [ ]:
## Create the inventory containing all the time series

# - files is the list of data files to load
# - pattern is applied to each data file to extract the strings {site} and {channel}
# - tag_template create the tag for the data file using {site} and {channel}
files = glob.glob("*/site*/*/*.ats")
pattern = "**/site{site}/*/*_T{channel}_*.ats"
tag_template = "site{site}_{channel}"

# correcting incorrect information about calibration files in file headers
name_converter = {
    'UNKN_H104.txt': 'MFS07104.txt',
    'UNKN_H105.txt': 'MFS07105.txt',
}
files

In [ ]:
# creating and filling the inventory
inv = rb.Inventory()
for fname, [tag] in rb.utils.tags_from_path(files, pattern, tag_template):
    calib = calibration(fname, name_converter)  # getting calibration for data file
    signal = rb.io.ats.load_ats([fname], [calib], lazy=True)  # loading data file
    inv.append(rb.SignalSet({tag:0}, signal))  # tagging and storing the signal

All the data are now loaded in `inventory`.
We can use `inventory` to explore and handle the data set.

You can check the number of files in your inventory `inv`:

In [ ]:
len(inv)

You can display the tags/labels included in the inventory `inv`

In [ ]:
inv.tags

Creating (using `pack()`) and showing (using `print()`) the SignalSet object for site004 only:

In [ ]:
print(inv.filter('site004*').pack())

Same operation for site099 (magnetic remote reference only):

In [ ]:
print(inv.filter('site099*').pack())

Creating and showing the SignalSet object content for the full inventory (including sites 002, 004, 006, 009, 100, 099).
The full data set is reduced to maximal synchronous time section. The `pack()` function is narrowing the time range to the window of common synchronousness of the whole inventory.

In [ ]:
print(inv.pack())

In [ ]:
# Function to prepare signal set from inventory to get it ready for TF estimation procedure
from itertools import chain

def prepare_signalset(inventory, local_site, remote_sites):
    patterns = (f"{e}*" for e in [local_site, *remote_sites])
    signalset = inventory.filter(*patterns).pack()
    tags = signalset.tags
    tags["E"] = tags[f"{local_site}_Ex"] + tags[f"{local_site}_Ey"]
    tags["B"] = tags[f"{local_site}_Hx"] + tags[f"{local_site}_Hy"]
    if remote_sites:
        remote_names = tags.filter(*chain(*(
            (f"{e}_Hx", f"{e}_Hy") for e in remote_sites
        )))
        tags["Bremote"] = sum((tags[n] for n in remote_names), ())
    return signalset

The function `prepare_signalset()` build the SignalSet needed for processing a given `local_site` along with some `remote_sites`.

The function starts by extracting the channels of interest and `pack` them in a SignalSet.
Then that signaset is enriched with specific tags (`'E'`, `'B'` and `'Bremote'`) that will be used later in the TF estimate function.

Showing the SignalSet object with `'E'`, `'B'` and `'Bremote'` tags for processing site004 using sites 100 and 099 as remote references

In [ ]:
print(prepare_signalset(inv, 'site004', ['site100', 'site099']))

Defining a frequency array in logscale for TF computation

In [ ]:
# Definining your output frequency in logscale / you can reduce nb_freq if you want to make a quick test
# as sampling frequency is 128, we go up to half a nyquist frequency which is 32 Hz
# recordings are long enough to try to reach 1 mHz
nb_freq=32
freq = np.logspace(-3, np.log10(32), nb_freq)
print(freq)

### Computing two-stage OLS Impedance estimate for site004 with sites 100 and 99 as remote references 

First stage: a regression is performed to estimate the TF between magnetic field at site 4 and magnetic field at (sites 99 + 100) . Second Stage: the first stage TF is used to produce a synthetic magnetic field and a second regression is operated between the latter and site 4 electric field.  

In [ ]:
sig = prepare_signalset(inv, 'site004', ['site100', 'site099'])
print(sig)
ImpOLS = rb.utils.impedance(sig, freq ,remote='Bremote' )
print(ImpOLS.impedance.shape)

Showing typical apparent resistivity and phase results using matplotlib

In [ ]:
res = ImpOLS
rho = 1e12 * np.abs(res.impedance)**2 / freq[:, None, None]
rho_err = 1e12 * np.abs(res.error)**2 / freq[:, None, None]
phi = np.angle(res.impedance, deg=True)
rad_err = np.arcsin(res.error/abs(res.impedance))
rad_err[np.isnan(rad_err)] = np.pi
phi_err = np.rad2deg(rad_err)

fig = plt.figure()
ax = plt.subplot(2, 1, 1)
ax.set_xscale("log", nonpositive='clip')
ax.set_yscale("log", nonpositive='clip')
ax.errorbar(freq, rho[:,0,0], yerr=rho_err[:,0,0], fmt='k.', label=r'$\rho_{xx}$')
ax.errorbar(freq, rho[:,1,1], yerr=rho_err[:,1,1], fmt='g.', label=r'$\rho_{yy}$')
ax.errorbar(freq, rho[:,0,1], yerr=rho_err[:,0,1], fmt='r.', label=r'$\rho_{xy}$')
ax.errorbar(freq, rho[:,1,0], yerr=rho_err[:,1,0], fmt='b.', label=r'$\rho_{yx}$')
plt.xlabel('freq')
plt.ylabel(r'apparent resistivity  $\rho$ ($\Omega.m$)');
plt.legend()

plt.title('Site 002 results in 2-stage RR OLS\n  configuration with sites 99 and 100 as remote references')
ax = plt.subplot(2, 1, 2)
ax.set_xscale("log", nonpositive='clip')
ax.errorbar(freq, phi[:,0,0], yerr=phi_err[:,0,0], fmt='k.', label=r'$\phi_{xx}$')
ax.errorbar(freq, phi[:,1,1], yerr=phi_err[:,1,1], fmt='g.', label=r'$\phi_{yy}$')
ax.errorbar(freq, phi[:,0,1], yerr=phi_err[:,0,1], fmt='r.', label=r'$\phi_{xy}$')
ax.errorbar(freq, phi[:,1,0], yerr=phi_err[:,1,0], fmt='b.', label=r'$\phi_{yx}$')
plt.xlabel('freq')
plt.ylabel(r'phase $\phi$ (degrees)');
plt.legend()
plt.ylim(-180, 180)


### Now computing 2-stage M-Estimator Transfer Function for site004 with sites 100 and 99 as remote references 

In [ ]:
from razorback.weights import mest_weights
from razorback.prefilters import cod_filter

sig = prepare_signalset(inv, 'site004', ['site100', 'site099'])
print(sig)
ImpME = rb.utils.impedance(
    sig, freq,
    weights= mest_weights,
    remote='Bremote', # including the remotes references in the computation,
    prefilter=cod_filter(0.0), # no coherency prefilter...
    fourier_opts=dict( Nper= 8,  overlap= 0.71) # fourier options with 8 periods by window, and 71% of overlap
)
print(ImpME.impedance.shape)

In [ ]:
res = ImpME
rho = 1e12 * np.abs(res.impedance)**2 / freq[:, None, None]
rho_err = 1e12 * np.abs(res.error)**2 / freq[:, None, None]
phi = np.angle(res.impedance, deg=True)
rad_err = np.arcsin(res.error/abs(res.impedance))
rad_err[np.isnan(rad_err)] = np.pi
phi_err = np.rad2deg(rad_err)

fig = plt.figure()
ax = plt.subplot(2, 1, 1)
ax.set_xscale("log", nonpositive='clip')
ax.set_yscale("log", nonpositive='clip')
ax.errorbar(freq, rho[:,0,0], yerr=rho_err[:,0,0], fmt='k.', label=r'$\rho_{xx}$')
ax.errorbar(freq, rho[:,1,1], yerr=rho_err[:,1,1], fmt='g.', label=r'$\rho_{yy}$')
ax.errorbar(freq, rho[:,0,1], yerr=rho_err[:,0,1], fmt='r.', label=r'$\rho_{xy}$')
ax.errorbar(freq, rho[:,1,0], yerr=rho_err[:,1,0], fmt='b.', label=r'$\rho_{yx}$')
plt.xlabel('freq')
plt.ylabel(r'apparent resistivity  $\rho$ ($\Omega.m$)');
plt.legend()

plt.title('Site 002 results in 2-stage RR ME\n  configuration with sites 99 and 100 as remote references')
ax = plt.subplot(2, 1, 2)
ax.set_xscale("log", nonpositive='clip')
ax.errorbar(freq, phi[:,0,0], yerr=phi_err[:,0,0], fmt='k.', label=r'$\phi_{xx}$')
ax.errorbar(freq, phi[:,1,1], yerr=phi_err[:,1,1], fmt='g.', label=r'$\phi_{yy}$')
ax.errorbar(freq, phi[:,0,1], yerr=phi_err[:,0,1], fmt='r.', label=r'$\phi_{xy}$')
ax.errorbar(freq, phi[:,1,0], yerr=phi_err[:,1,0], fmt='b.', label=r'$\phi_{yx}$')
plt.xlabel('freq')
plt.ylabel(r'phase $\phi$ (degrees)');
plt.legend()
plt.ylim(-180, 180)


### Now computing 2-stage Bounded Influence Transfer Function for site004 with sites 100 and 99 as remote references for a rejection percentage of 1% and 3 bounded influence steps

In [ ]:
from razorback.weights import bi_weights
from razorback.prefilters import cod_filter

sig = prepare_signalset(inv, 'site004', ['site100', 'site099'])
print(sig)
ImpBI = rb.utils.impedance(
    sig, freq,
     weights= bi_weights(0.01, 3),  # bounded influence with reject probability of 1% and 3 steps
    remote='Bremote', # including the remotes references in the computation,
    prefilter=cod_filter(0.0), # prefilter: cod_filter(0.0)
    fourier_opts=dict( Nper= 8,  overlap= 0.71) # fourier options with 8 periods by window, and 71% of overlap
)
print(ImpBI.impedance.shape)

In [ ]:
res = ImpBI
rho = 1e12 * np.abs(res.impedance)**2 / freq[:, None, None]
rho_err = 1e12 * np.abs(res.error)**2 / freq[:, None, None]
phi = np.angle(res.impedance, deg=True)
rad_err = np.arcsin(res.error/abs(res.impedance))
rad_err[np.isnan(rad_err)] = np.pi
phi_err = np.rad2deg(rad_err)

fig = plt.figure()
ax = plt.subplot(2, 1, 1)
ax.set_xscale("log", nonpositive='clip')
ax.set_yscale("log", nonpositive='clip')
ax.errorbar(freq, rho[:,0,0], yerr=rho_err[:,0,0], fmt='k.', label=r'$\rho_{xx}$')
ax.errorbar(freq, rho[:,1,1], yerr=rho_err[:,1,1], fmt='g.', label=r'$\rho_{yy}$')
ax.errorbar(freq, rho[:,0,1], yerr=rho_err[:,0,1], fmt='r.', label=r'$\rho_{xy}$')
ax.errorbar(freq, rho[:,1,0], yerr=rho_err[:,1,0], fmt='b.', label=r'$\rho_{yx}$')
plt.xlabel('freq')
plt.ylabel(r'apparent resistivity  $\rho$ ($\Omega.m$)');
plt.legend()

plt.title('Site 002 results in 2-stage RR BOUNDED INFLUENCE \n  configuration with sites 99 and 100 as remote references')
ax = plt.subplot(2, 1, 2)
ax.set_xscale("log", nonpositive='clip')
ax.errorbar(freq, phi[:,0,0], yerr=phi_err[:,0,0], fmt='k.', label=r'$\phi_{xx}$')
ax.errorbar(freq, phi[:,1,1], yerr=phi_err[:,1,1], fmt='g.', label=r'$\phi_{yy}$')
ax.errorbar(freq, phi[:,0,1], yerr=phi_err[:,0,1], fmt='r.', label=r'$\phi_{xy}$')
ax.errorbar(freq, phi[:,1,0], yerr=phi_err[:,1,0], fmt='b.', label=r'$\phi_{yx}$')
plt.xlabel('freq')
plt.ylabel(r'phase $\phi$ (degrees)');
plt.legend()
plt.ylim(-180, 180)